<a href="https://colab.research.google.com/github/NisargShah1/GenAI/blob/main/prompt_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!pip install accelerate

In [3]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HUG_KEY'))

In [4]:
from transformers import AutoTokenizer
import transformers
import torch

In [5]:
#specify the model you want to use
model = "meta-llama/Llama-3.1-8B"

In [6]:
#load the pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
#make it ready for text generation
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
!pip install PyPDF2 requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.1 MB/s eta 0:00:00


In [9]:
import requests
import PyPDF2
from io import BytesIO

def download_pdf_from_url(url):
    # Send a GET request to fetch the PDF
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Return the content as a BytesIO object (in-memory binary stream)
        return BytesIO(response.content)
    else:
        print(f"Failed to download PDF, status code: {response.status_code}")
        return None

def extract_text_from_pdf(pdf_stream):
    # Read the PDF content from the in-memory binary stream
    reader = PyPDF2.PdfReader(pdf_stream)
    text = ""

    # Extract text from each page
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()

    return text

# Example usage
pdf_url = 'https://mf.nipponindiaim.com/FundsAndPerformance/ProductNotes/NipponIndia-Short-Term-Fund-Jan-2024.pdf'
pdf_stream = download_pdf_from_url(pdf_url)

if pdf_stream:
    pdf_text = extract_text_from_pdf(pdf_stream)
    print(pdf_text)


Investment Horizon: 1 - 3 years
Contact your Mutual Fund Distributor  |  Call 1860 266 0111#, 
91-22-6925 9696# (For investors outside India)  |
Visit mf.nipponindiaim.com
#Charges applicable.Nippon India 
Short Term Fund
(An open  ended  short  term  debt  scheme  investing  in 
instruments  such  that  the Macaulay  duration  of the 
portfolio  is between  1 to 3 years . Relatively  High  interest  
rate  risk and moderate  Credit  Risk)
Mutual Fund investments are subject to market risks, read all scheme related documents carefully.This product is suitable for investors who are seeking*
•Income over short term
•Investment in debt & money market instruments with 
portfolio Macaulay Duration between 1 -3 years
*Investors should consult their financial advisors 
if in doubt about  whether the product is suitable 
for them.Benchmark RiskometerNippon India Short Term 
FundCRISIL Short Duration Debt 
B-II Index
Investors understand that their principal  
will be at Moderate riskBenchmark 

In [29]:
user_prompt = 'what is nippon india short term fund?'

In [30]:
full_prompt = f'''Use the context given below for answering the question.

question: {user_prompt}
context:  {pdf_text}

'''

In [31]:
messages = [{'role':'system','content':'I want you to act like a chatbot for Nippon India short term fund 2024.'},
           {"role":"user","content":full_prompt}]
response = pipeline(messages)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [32]:
print(response[0]["generated_text"][-1])

{'role': 'assistant', 'content': '. The 1.5-liter gasoline engine with 110 hp, 5-speed manual transmission and'}
